In [1]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

# tf.enable_eager_execution()
tf_config = tf.ConfigProto()
tf_config.gpu_options.allow_growth = True
sess = tf.Session(config=tf_config)
import sys
from pathlib import Path

import keras
import pandas as pd
from keras import backend as K
from keras import layers, models, optimizers, regularizers
from keras.initializers import Constant
from keras.callbacks import CSVLogger, LearningRateScheduler, ModelCheckpoint
from keras.datasets import mnist
from keras.layers import (
    Activation,
    BatchNormalization,
    Concatenate,
    Conv1D,
    Conv2D,
    Conv2DTranspose,
    Dense,
    Dropout,
    Flatten,
    Input,
    Lambda,
    Layer,
    MaxPooling2D,
    PReLU,
    Reshape,
    Softmax,
)
from keras.utils import to_categorical

keras.__version__

Using TensorFlow backend.


'2.3.1'

In [2]:
path_photoz = "/home/bid13/code/photozCapsNet"

sys.path.insert(1, path_photoz)
path_photoz = Path(path_photoz)

In [3]:
import encapzulate
from encapzulate.base.deepCapsLayers import (
    CapsToScalars,
    CapsuleLayer,
    Conv2DCaps,
    ConvCapsuleLayer3D,
    ConvertToCaps,
    FlattenCaps,
    Mask_CID,
)
from encapzulate.base.loss import margin_loss
from encapzulate.data_loader.data_loader import load_data
from encapzulate.models.multi_gpu import MultiGPUModel
from encapzulate.utils import metrics
from encapzulate.utils.fileio import load_config, load_model
from encapzulate.utils.metrics import Metrics, bins_to_redshifts, probs_to_redshifts

In [4]:
from importlib import reload
import normalization
reload(normalization)
from normalization import LayerNormalization

In [5]:
config = load_config(path_photoz / "encapzulate" / "configs" / "morphCaps.yml")
config["run_name"] = 'morphCapsDeep_multi_val_0'
# config["dataset"] = "sdss_GZ_weighted_regression"
# config["path_data"] = "/data/bid13/photoZ/data/pasquet2019"
config["input_shape"] = config["image_shape"]
# config["run_name"] = "test"
# config["reg_weight"] = 1.0
config["epochs"] = 4
# config["num_class"] = 3
# config["learning_rate"] = 0.001
# config["decay_rate"] = 0.97


config:
{   'bands': ('u', 'g', 'r', 'i', 'z'),
    'batch_size': 400,
    'checkpoint': None,
    'compile_on': 'cpu',
    'dataset': 'sdss_gz1_iter1',
    'decay_rate': 0.95,
    'dim_capsule': 16,
    'epochs': 100,
    'frac_dev': 0.1,
    'frac_train': 0.2,
    'image_scale': 10,
    'image_shape': (64, 64, 5),
    'img_augmentation': 2,
    'lam_recon': 0.005,
    'lam_redshift': 1,
    'learning_rate': 0.001,
    'logistic': True,
    'model_name': 'morphCapsDeep_2',
    'num_class': 2,
    'num_gpus': 2,
    'num_quantiles': False,
    'path_data': '/data/bid13/photoZ/data/pasquet2019',
    'path_results': None,
    'random_state': 200,
    'routings': 3,
    'run_name': 'morphCapsDeep_multi_val_5',
    'timeline': False,
    'use_vals': False,
    'z_max': 0.4,
    'z_min': 0.0}



In [6]:
path_output = "/home/bid13/code/photozCapsNet/results"
path_output = Path(path_output)
path_results = (
    path_output / config["run_name"].split("_")[0] / config["run_name"] / "results"
)
path_logs = path_results / "logs"
path_weights = path_results / "weights"
path_logs.mkdir(parents=True, exist_ok=True)
path_weights.mkdir(parents=True, exist_ok=True)

In [7]:
(
    (x_train, y_train, vals_train, z_spec_train, cat_train),
    (x_dev, y_dev, vals_dev, z_spec_dev, cat_dev),
    (x_test, y_test, vals_test, z_spec_test, cat_test),
) = load_data(load_cat=True, **config)

In [8]:
# mask_train = np.isfinite(vals_train).all(axis=1)
# mask_test = np.isfinite(vals_test).all(axis=1)
# mask_dev = np.isfinite(vals_dev).all(axis=1)

In [9]:
K.clear_session()

In [10]:
class CustomMultiLossLayer(layers.Layer):
    def __init__(self, **kwargs):
        self.is_placeholder = True
        super(CustomMultiLossLayer, self).__init__(**kwargs)

    def build(self, input_shape=None):
        # initialise log_vars
        self.hinge_log_var = self.add_weight(
            name="hinge_log_var", shape=(1,), initializer=Constant(0.0), trainable=True
        )
        self.decoder_log_var = self.add_weight(
            name="decoder_log_var",
            shape=(1,),
            initializer=Constant(0.0),
            trainable=True,
        )
        self.redshift_log_var = self.add_weight(
            name="redshift_log_var",
            shape=(1,),
            initializer=Constant(0.0),
            trainable=True,
        )

        super(CustomMultiLossLayer, self).build(input_shape)

    def multi_loss(self, x_true, y_true, z_true, x_pred, y_pred, z_pred):
        hinge_ivar = K.exp(-self.hinge_log_var)
        loss = K.sum(
            hinge_ivar * margin_loss(y_true, y_pred) + self.hinge_log_var, axis=-1
        )

        decoder_ivar = K.exp(-self.decoder_log_var)
        loss += K.sum(
            decoder_ivar * K.sum((x_true - x_pred) ** 2.0) + self.decoder_log_var, axis=-1
        )

        redshift_ivar = K.exp(-self.redshift_log_var)
        loss += K.sum(
            redshift_ivar * (z_true - z_pred) ** 2.0 + self.redshift_log_var, axis=-1
        )
        

        return K.mean(loss)

    def call(self, inputs):
        x_true, y_true, z_true, x_pred, y_pred, z_pred = inputs
        loss = self.multi_loss(x_true, y_true, z_true, x_pred, y_pred, z_pred)
        self.add_loss(loss, inputs=inputs)
        # We won't actually use the output.
        return z_pred

In [11]:
def CapsNet(input_shape, num_class, routings, dim_capsule, **kwargs):
    # assemble encoder
    x = Input(shape=input_shape)
    l = x

    l = Conv2D(
        128,
        (3, 3),
        strides=(1, 1),
        activation="relu",
        padding="same",
        kernel_initializer="he_normal",
    )(
        l
    )  # common conv layer
    l = BatchNormalization()(l)
    l = ConvertToCaps()(l)

    l = Conv2DCaps(
        32, 4, kernel_size=(3, 3), strides=(2, 2), r_num=1, b_alphas=[1, 1, 1]
    )(l)
    l_skip = Conv2DCaps(
        32, 4, kernel_size=(3, 3), strides=(1, 1), r_num=1, b_alphas=[1, 1, 1]
    )(l)
    l = Conv2DCaps(
        32, 4, kernel_size=(3, 3), strides=(1, 1), r_num=1, b_alphas=[1, 1, 1]
    )(l)
    l = Conv2DCaps(
        32, 4, kernel_size=(3, 3), strides=(1, 1), r_num=1, b_alphas=[1, 1, 1]
    )(l)
    l = layers.Add()([l, l_skip])

    l = Conv2DCaps(
        32, 8, kernel_size=(3, 3), strides=(2, 2), r_num=1, b_alphas=[1, 1, 1]
    )(l)
    l_skip = Conv2DCaps(
        32, 8, kernel_size=(3, 3), strides=(1, 1), r_num=1, b_alphas=[1, 1, 1]
    )(l)
    l = Conv2DCaps(
        32, 8, kernel_size=(3, 3), strides=(1, 1), r_num=1, b_alphas=[1, 1, 1]
    )(l)
    l = Conv2DCaps(
        32, 8, kernel_size=(3, 3), strides=(1, 1), r_num=1, b_alphas=[1, 1, 1]
    )(l)
    l = layers.Add()([l, l_skip])

    l = Conv2DCaps(
        32, 8, kernel_size=(3, 3), strides=(2, 2), r_num=1, b_alphas=[1, 1, 1]
    )(l)
    l_skip = Conv2DCaps(
        32, 8, kernel_size=(3, 3), strides=(1, 1), r_num=1, b_alphas=[1, 1, 1]
    )(l)
    l = Conv2DCaps(
        32, 8, kernel_size=(3, 3), strides=(1, 1), r_num=1, b_alphas=[1, 1, 1]
    )(l)
    l = Conv2DCaps(
        32, 8, kernel_size=(3, 3), strides=(1, 1), r_num=1, b_alphas=[1, 1, 1]
    )(l)
    l = layers.Add()([l, l_skip])
    l1 = l

    l = Conv2DCaps(
        32, 8, kernel_size=(3, 3), strides=(2, 2), r_num=1, b_alphas=[1, 1, 1]
    )(l)
    l_skip = ConvCapsuleLayer3D(
        kernel_size=3,
        num_capsule=32,
        num_atoms=8,
        strides=1,
        padding="same",
        routings=3,
    )(l)
    l = Conv2DCaps(
        32, 8, kernel_size=(3, 3), strides=(1, 1), r_num=1, b_alphas=[1, 1, 1]
    )(l)
    l = Conv2DCaps(
        32, 8, kernel_size=(3, 3), strides=(1, 1), r_num=1, b_alphas=[1, 1, 1]
    )(l)
    l = layers.Add()([l, l_skip])
    l2 = l

    la = FlattenCaps()(l2)
    lb = FlattenCaps()(l1)
    l = layers.Concatenate(axis=-2)([la, lb])

    #     l = Dropout(0.4)(l)
    digits_caps = CapsuleLayer(
        num_capsule=num_class,
        dim_capsule=dim_capsule,
        routings=routings,
        channels=0,
        name="digit_caps",
    )(l)

    l = CapsToScalars(name="capsnet")(digits_caps)
    # l = Softmax()(l)

    m_capsnet = models.Model(inputs=x, outputs=l, name="capsnet_model")

    y = Input(shape=(num_class,))

    masked_by_y = Mask_CID()([digits_caps, y])
    masked = Mask_CID()(digits_caps)

    # Redshift Network
    z = Input(shape=(1,))
    redshift_input = Input(shape=(dim_capsule,))
    #     r = LayerNormalization()(redshift_input)
    #     r = BatchNormalization(momentum=0.9)(redshift_input)
    r = Dense(128, kernel_initializer="he_normal")(redshift_input)
    r = PReLU()(r)
    r = Dense(64, kernel_initializer="he_normal")(r)
    r = PReLU()(r)
    r = Dense(32, kernel_initializer="he_normal")(r)
    r = PReLU()(r)
    r = Dense(16, kernel_initializer="he_normal")(r)
    r = PReLU()(r)
    redshift_out = Dense(1)(r)
    #     redshift_out = Dense(1)(redshift_input)
    redshift = models.Model(redshift_input, redshift_out, name="redshift_model")

    # Decoder Network
    decoder_input = Input(shape=(dim_capsule,))
    d = Dense(np.prod(input_shape), kernel_initializer="he_normal",)(decoder_input)
    d = PReLU()(d)
    d = Reshape(input_shape)(d)

    d = Conv2DTranspose(64, (3, 3), padding="same", kernel_initializer="he_normal",)(d)
    d = PReLU()(d)
    d = Conv2DTranspose(32, (3, 3), padding="same", kernel_initializer="he_normal",)(d)
    d = PReLU()(d)
    d = Conv2DTranspose(16, (3, 3), padding="same", kernel_initializer="he_normal",)(d)
    d = PReLU()(d)
    d = Conv2DTranspose(8, (3, 3), padding="same", kernel_initializer="he_normal")(d)
    d = PReLU()(d)
    d = Conv2DTranspose(
        input_shape[-1],
        (3, 3),
        padding="same",
        activation="tanh",
        kernel_initializer="he_normal",
    )(d)
    decoder_output = Reshape(target_shape=input_shape, name="out_recon")(d)

    decoder = models.Model(decoder_input, decoder_output, name="decoder_model")

    loss_layer = CustomMultiLossLayer()(
        [x, y, z, decoder(masked_by_y), m_capsnet.output, redshift(masked_by_y)]
    )

    train_model = models.Model([x, y, z], [loss_layer],)

    # Add required metrics for monitoring training
    train_model.add_metric((z - redshift(masked)) ** 2, name="redshift_loss")
    if kwargs["logistic"]:
        inv_log_z_true = (K.exp(z) * kwargs["z_max"] + kwargs["z_min"]) / (K.exp(z) + 1)
        inv_log_z_pred = (
            K.exp(redshift(masked)) * kwargs["z_max"] + kwargs["z_min"]
        ) / (K.exp(redshift(masked)) + 1)
        train_model.add_metric(
            (inv_log_z_true - inv_log_z_pred) ** 2, name="redshift_mse"
        )
    train_model.add_metric((x - decoder(masked)) ** 2, name="decoder_mse")
    train_model.add_metric(
        tf.cast(
            tf.equal(
                tf.compat.v1.argmax(y, axis=-1),
                tf.compat.v1.argmax(m_capsnet.output, axis=-1),
            ),
            K.floatx(),
        ),
        name="accuracy",
    )

    eval_model = models.Model(
        [x,],
        [masked, digits_caps, m_capsnet.output, decoder(masked), redshift(masked),],
    )

    manipulate_model = models.Model(
        [x,], [masked, m_capsnet.output, decoder(masked), redshift(masked)],
    )
    train_model.summary()

    return train_model, eval_model, manipulate_model, decoder, redshift

In [12]:
train_model, eval_model, manipulate_model, decoder, redshift_model = CapsNet(**config)
parallel_train_model = MultiGPUModel(train_model, gpus=2)
train_model = parallel_train_model

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Tensor("conv_capsule_layer3d_1/stack:0", shape=(5,), dtype=int32)
Instructions for updating:
dim is deprecated, use axis instead
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 64, 64, 5)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 64, 64, 128)  5888        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 64, 64, 128)  512         conv2d_1[0][0]                   
________________________________________________________________________________

In [13]:
compile_kwargs = {
    "optimizer": optimizers.Adam(lr=config["learning_rate"]),
#     "loss": None,#[margin_loss, "mse", "mse"],
#     "loss_weights": None, #[
# #         1.0,
# #         config["lam_recon"] * np.prod(config["input_shape"]),
# #         config["lam_redshift"],
# #     ],
#      "metrics": {"capsnet": "accuracy",},
}
train_model.compile(**compile_kwargs)

/opt/anaconda3/envs/tf15/lib/python3.6/site-packages/keras/engine/training_utils.py:819: UserWarning: Output custom_multi_loss_layer_1 missing from loss dictionary. We assume this was done on purpose. The fit and evaluate APIs will not be expecting any data to be passed to custom_multi_loss_layer_1.
  'be expecting any data to be passed to {0}.'.format(name))


In [ ]:
# Train model on dataset
lr_decay = LearningRateScheduler(
    schedule=lambda epoch: config["learning_rate"] * (config["decay_rate"] ** epoch)
)
log = CSVLogger(str(path_logs / "log.csv"))
cp = ModelCheckpoint(
    filepath=str(path_weights / "weights-{epoch:02d}.h5"),
    save_best_only=False,
    save_weights_only=True,
    verbose=1,
    mode="max",
)


train_model.fit(
    x=[x_train, y_train, z_spec_train],
    y = None,
    batch_size=config["batch_size"],
    validation_data=[[x_dev,y_dev, z_spec_train], None,],
    epochs=config["epochs"],
    initial_epoch=0,
    callbacks=[log, cp, lr_decay],
)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Train on 92171 samples, validate on 46086 samples
Epoch 1/4
92171/92171 [==============================] - 326s 4ms/step - loss: 19614.8566 - redshift_loss: 0.5352 - redshift_mse: 0.0029 - decoder_mse: 0.0025 - accuracy: 0.6526 - val_loss: 11601.1991 - val_redshift_loss: 0.6835 - val_redshift_mse: 0.0034 - val_decoder_mse: 0.0015 - val_accuracy: 0.6572

Epoch 00001: saving model to /home/bid13/code/photozCapsNet/results/morphCapsDeep/morphCapsDeep_multi_val_0/results/weights/weights-01.h5
Epoch 2/4
92171/92171 [==============================] - 317s 3ms/step - loss: 10095.9203 - redshift_loss: 0.5480 - redshift_mse: 0.0027 - decoder_mse: 0.0014 - accuracy: 0.3832 - val_loss: 10602.9034 - val_redshift_loss: 0.7389 - val_redshift_mse: 0.0035 - val_decoder_mse: 0.0015 - val_accuracy: 0.3233

Epoch 00002: saving model to /home/bid13/code/photozCapsNet/results/morphCapsDeep/morphCapsDeep_multi_val

In [ ]:
log = pd.read_csv(path_results / "logs" / "log.csv")
log